## Session 1: The List standard library

Modern programming languages often provide a standard library of commonly used data structures, functions and elementary operations, so that programmers do not need to re-implement these features themselves. One should consider using standard library functions because they are available across all implementations and (usually) well-suited for optimizations.

Today, we look at the standard library for [Lists](https://ocaml.org/manual/5.2/api/List.html).
The library functions are given in the following format, showing its name, type, and a short description.

> `val length : 'a list -> int`
>
> &emsp; Return the length (number of elements) of the given list.

To warm up, let's try to write the (non tail-recursive) `length` function.

val length : 'a list -> int = <fun>


Now, the tail-recursive version of `length`. Note that we do not need to know the value of the head, so we could just place an underscore "wildcard" there.

How do we know if a library function is tail-recursive or not? Check the descriptions:

> Some functions are flagged as not tail-recursive. A tail-recursive function uses constant stack space, while a non-tail-recursive function uses stack space proportional to the length of its list argument, which can be a problem with very long lists. When the function takes several list arguments, an approximate formula giving stack usage (in some unspecified constant unit) is shown in parentheses.
> 
> The above considerations can usually be ignored if your lists are not longer than about 10000 elements.

There is no flag on the `length` function, it means that `List.length` is tail recursive. Most functions in the library are tail recursive, for example, `rev`.

> `rev : 'a list -> 'a list`
>
> &emsp; List reversal.

The classic example of a non tail-recursive function is `append`.

> `append : 'a list -> 'a list -> 'a list`
> 
> &emsp; Concatenate two lists. Same function as the infix operator @. Not tail-recursive (length of the first argument). The @ operator is not tail-recursive either.

In OCaml, we can define infix operators like this:

We have implemented some functions, and now is a good time to test them. We can use `init` to generate some test lists.

> `init : int -> (int -> 'a) -> 'a list`
>
> &emsp; `init len f` is `[f 0; f 1; ...; f (len-1)]`, evaluated left to right.

Or, the tail-recursive version, using reverse.

Time for some tests!

In [ ]:
let l1 = init 10 (fun x -> x)

In [ ]:
let l2 = init 5 (fun x -> 10 + x)

In [ ]:
length (rev (l1 @ l2))

Now, we look at iterators, i.e. ways to traverse every element of a list.

> ` map : ('a -> 'b) -> 'a list -> 'b list`
>
> &emsp; `map f [a1; ...; an]` applies function `f` to `a1, ..., an`, and builds the list `[f a1; ...; f an]` with the results returned by `f`.

We can, of course, implement the tail-recursive version of map using an accumulator and reverse.

But, as you can see, we pay an extra time to reverse the list, which is not desirable. Both implementations have $O(n)$ time complexity, but the tail-recursive one has a larger constant factor.
OCaml's standard library uses a technique called *tail recursion modulo cons* to avoid the extra time due to reverse.

Now we look at `mapi`, a more advanced version of `map` such that `f` has access to the element's current position.

> `mapi : (int -> 'a -> 'b) -> 'a list -> 'b list`
>
> &emsp; Same as List.map, but the function is applied to the index of the element as first argument (counting from 0), and the element itself as second argument.

Reverse map is naturally tail-recursive.

> `val rev_map : ('a -> 'b) -> 'a list -> 'b list`
>
> &emsp; `rev_map f l` gives the same result as `List.rev (List.map f l)`, but is more efficient.


Another way of going through a list is summing.

In general, this is an operation called folding (order of evaluation is left to right).

> `fold_left : ('acc -> 'a -> 'acc) -> 'acc -> 'a list -> 'acc`
> 
> &emsp; `fold_left f init [b1; ...; bn]` is `f (... (f (f init b1) b2) ...) bn`.

We get `sum` by setting `f = (+)` and `init = 0`.

We may also fold with order from right to left.

> `fold_right : ('a -> 'acc -> 'acc) -> 'a list -> 'acc -> 'acc`
>
> &emsp; `fold_right f [a1; ...; an] init` is `f a1 (f a2 (... (f an init) ...))`. Not tail-recursive.

We can write some tests with the string concatenation operator `(^)`.

In [ ]:
let strs = ["a"; "b"; "c"; "d"; "e"]

In [ ]:
fold_left (^) "x" strs

In [ ]:
fold_right (^) strs "x"

Finally, we look at functions for list scanning, i.e. finding elements or elements with special properties in the list.

> `mem : 'a -> 'a list -> bool`
>
> &emsp; `mem a set` is true if and only if `a` is equal to an element of `set`.

The logical "or" operator `||` is a more concise way of writing `if a = x then true else mem a xs`, since `a or b` is always true if `a` is true, and if `a` is false, its value is the same as `b`. Similarly, we may write `a && b`, the logical `a and b` for `if a then b else false`.
We use this in the implementation of `for_all` and `exists`.
> `for_all : ('a -> bool) -> 'a list -> bool`
> 
> &emsp; `for_all f [a1; ...; an]` checks if all elements of the list satisfy the predicate `f`. That is, it returns `(f a1) && (f a2) && ... && (f an)` for a non-empty list and `true` if the list is empty.

> `exists : ('a -> bool) -> 'a list -> bool`
> 
> &emsp; `exists f [a1; ...; an]` checks if at least one element of the list satisfies the predicate `f`. That is, it returns `(f a1) || (f a2) || ... || (f an)` for a non-empty list and false if the list is empty.

### Warm-up exercise

**Recursion**

1. Write the factorial function so that `factorial n` calculates `n * (n-1) * (n-2) * ... * 1`.

2. Write a second version of `factorial` in tail-recursive style.


3. Using subtraction, write the remainder function so that `remainder x n` calculates the remainder when `n` is divided by `x`.

4. How many steps does your implementation of `remainder` take for inputs `x` and `n`?

5. Write the greatest common divisor function so that `gcd x y` finds the highest integer that exactly divides both `x` and `y`.

**Booleans**

1. Using `if...then...else`, `false` and `true`, define the boolean functions:

- `not`
- `both` (logical _and_)
- `either` (logical _or_)
- `xor` (logical _exclusive or_)

**Characters**

1. Using OCaml's comparison operators `<=`, `<`, `>=`, `>`, `=`, define the functions

- `is_upper` that checks whether a character is an uppercase character ('A' to 'Z')
- `is_lower` that checks whether a character is a lower character ('a' to 'z')
- `is_digit` that checks whether a character is a digit ('0' to '9')
- `is_alpha` that checks whether a character is an alphabetic character ('a' to 'z' or 'A' to 'Z')
- `is_vowel` that checks whether a character is a vowel
